In [0]:
import dlt
import json
from pyspark.sql.functions import col, regexp_replace, from_json
from pyspark.sql.types import ArrayType, StringType, DoubleType

# Získání prostředí (např. "dev" nebo "prod")
env = spark.conf.get("pipeline.env")
catalog = "principal_lab_db"
silver_schema = f"{env}_silver"
bronze_schema = f"{env}_bronze"

# Nastavení katalogu
spark.sql(f"USE CATALOG {catalog}")
spark.sql(f"USE SCHEMA {silver_schema}")

# Načtení metadata z lookup tabulky
config = spark.table(f"{catalog}.config_{env}.table_lookup") \
    .filter(col("table_name") == "dim_policies") \
    .select("keys", "scd_type", "description") \
    .first()

# Parsování hodnot
keys_raw = config["keys"]
scd_type_raw = config["scd_type"]
description = config["description"]

# Pokud je 'keys' uložen jako JSON řetězec, tak ho rozparsuj
if isinstance(keys_raw, str):
    business_keys = json.loads(keys_raw)
else:
    business_keys = keys_raw

# Mapa pro SCD typ
scd_type_map = {
    "SCD1": 1,
    "SCD2": 2,
    "1": 1,
    "2": 2,
    None: 0,
    "": 0
}
scd_type = scd_type_map.get(str(scd_type_raw).upper(), 0)

# Debug pro sebe
print(f"Prostředí: {env}")
print(f"Business Keys: {business_keys}")
print(f"SCD Type: {scd_type}")

# Vytvoření view z Bronze vrstvy a čištění
@dlt.view(name="policies_bronze_clean")
def policies_bronze_clean():
    bronze_df = spark.readStream.table(f"{catalog}.{bronze_schema}.policies_bronze")

    cleaned_df = bronze_df \
        .withColumn("premium", col("premium").cast(DoubleType())) \
        .drop("_rescued_data", "source_file", "ingestion_ts") 

    return cleaned_df



# Silver tabulka se SCD2 historií
if scd_type == 2:
    # Tabulka dim_policies ve stříbrné vrstvě (SCD2)
    dlt.create_streaming_table(
        name="dim_policies",
        comment=description,
        table_properties={"quality": "silver"}
    )

    #  tady se vytváří historie přes __START_AT a __END_AT
    dlt.apply_changes(
        target="dim_policies",
        source="policies_bronze_clean",
        keys=business_keys,
        sequence_by=col("snapshot_date"),
        ignore_null_updates=False,
        stored_as_scd_type="2",
        track_history_except_column_list=["snapshot_date"]
    )

else:
    raise ValueError(f"Nepodporovaný nebo chybějící SCD typ: {scd_type_raw}")
